In [1]:
import pandas as pd
import requests
from itertools import combinations

In [2]:
api_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    print("Succesfully fetched data!")
else:
    print(f"Error while fetching data. Status code: {response.status_code}")

Succesfully fetched data!


In [3]:
print(data.keys())

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


In [4]:
players = pd.DataFrame(data['elements'])
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,137,82,674,304,682,310,636,278,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-2,2,0,...,32,15,303,38,459,49,213,37,0.74,0.37
2,True,True,75.0,75.0,226597,0,0,2,-2,2,...,66,3,76,17,31,4,13,5,1.08,0.33
3,True,True,100.0,100.0,219847,0,0,-1,1,1,...,13,5,88,18,37,14,37,10,1.01,0.29
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,638,63,425,51,514,58,558,68,0.00,0.00


In [5]:
print("Players DataFrame Columns:")
print(players.columns.tolist())


Players DataFrame Columns:
['can_transact', 'can_select', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'removed', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'region', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_

In [7]:
position_map = {
    1: 'GK',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
}

players_selected = players[
    ['first_name', 'second_name', 'team_code', 'element_type', 'total_points', 'points_per_game', 'now_cost']
].copy()

players_selected['now_cost'] = players_selected['now_cost'] / 10

players_selected['element_type'] = players_selected['element_type'].map(position_map)

players_selected[['total_points', 'points_per_game', 'now_cost']] = players_selected[
    ['total_points', 'points_per_game', 'now_cost']
].apply(pd.to_numeric, errors='coerce')

players_selected = players_selected.loc[players_selected['points_per_game'] > 0.0]

players_selected['full_name'] = players_selected['first_name'] + ' ' + players_selected['second_name']

players_selected['Cost/Points'] = (players_selected['now_cost'] / players_selected['total_points']).round(2)

print(players_selected)


         first_name             second_name  team_code element_type  \
1           Gabriel       Fernando de Jesus          3      Forward   
2           Gabriel    dos Santos Magalhães          3     Defender   
3               Kai                 Havertz          3      Forward   
5           Jurriën                  Timber          3     Defender   
6        Jorge Luiz            Frello Filho          3   Midfielder   
..              ...                     ...        ...          ...   
686          Jørgen           Strand Larsen         39      Forward   
687    Toti António                   Gomes         39     Defender   
689           André  Trindade da Costa Neto         39   Midfielder   
690  Carlos Roberto            Forbs Borges         39   Midfielder   
691           Alfie                    Pond         39     Defender   

     total_points  points_per_game  now_cost                     full_name  \
1               9              0.8       6.8     Gabriel Fernando de 

In [8]:
max_indeces = players_selected[['total_points', 'points_per_game', 'now_cost']].idxmax()
max_rows = players_selected.loc[max_indeces]
print(max_rows)


    first_name second_name  team_code element_type  total_points  \
401    Mohamed       Salah         14   Midfielder           151   
401    Mohamed       Salah         14   Midfielder           151   
428     Erling     Haaland         43      Forward            98   

     points_per_game  now_cost       full_name  
401             10.8      13.3   Mohamed Salah  
401             10.8      13.3   Mohamed Salah  
428              6.5      14.9  Erling Haaland  


In [9]:
defenders = players_selected[players_selected['element_type'] == 'Defender']

defenders.head()

,first_name,second_name,team_code,element_type,total_points,points_per_game,now_cost,full_name
2,Gabriel,dos Santos Magalhães,3,Defender,60,4.6,6.2,Gabriel dos Santos Magalhães
5,Jurriën,Timber,3,Defender,47,3.6,5.7,Jurriën Timber
7,Jakub,Kiwior,3,Defender,13,1.6,4.8,Jakub Kiwior
14,William,Saliba,3,Defender,55,3.9,6.2,William Saliba
17,Tomiyasu,Takehiro,3,Defender,1,1.0,4.8,Tomiyasu Takehiro


In [10]:
print(f"Total defenders: {len(defenders)}")

Total defenders: 160


In [11]:
min_indeces = defenders[['total_points', 'points_per_game', 'now_cost']].idxmin()
min_rows = defenders.loc[min_indeces]
print(min_rows)


    first_name second_name  team_code element_type  total_points  \
17    Tomiyasu    Takehiro          3     Defender             1   
170       Adam     Webster         36     Defender             1   
80       James        Hill         91     Defender             1   

     points_per_game  now_cost          full_name  
17               1.0       4.8  Tomiyasu Takehiro  
170              0.3       4.3       Adam Webster  
80               1.0       3.9         James Hill  


In [12]:
# Generate all combinations of 4 defenders
defender_combinations = list(combinations(defenders.itertuples(), 4))

print(len(defender_combinations))

26294360


In [14]:
optimal_defenders = None
best_total_points = 0
best_comb_cost = 0
best_cpp = 100
max_cost = 50

for player1, player2, player3, player4 in defender_combinations:
    total_cost = player1.now_cost + player2.now_cost + player3.now_cost + player4.now_cost
    if total_cost > max_cost:
        continue

    total_points = player1.total_points + player2.total_points + player3.total_points + player4.total_points
    cpp = total_cost / total_points

    if cpp < best_cpp:
        best_cpp = cpp
        best_total_points = total_points
        best_comb_cost = total_cost
        optimal_defenders = (player1, player2, player3, player4)

# Unpack and print the results
print(f"Total points: {best_total_points}")
print(f"Total cost: {best_comb_cost} \n")
print(f"Cost/point: {best_cpp} \n")

for player in optimal_defenders:
    print(f"Player: {player.first_name} {player.second_name} - {player.element_type} - Team Code: {player.team_code} - Total points: {player.total_points} - Cost: £{player.now_cost}m")


Total points: 218
Total cost: 18.6 

Cost/point: 0.0853211009174312 

Player: Ashley Young - Defender - Team Code: 11 - Total points: 57 - Cost: £4.6m
Player: Ola Aina - Defender - Team Code: 17 - Total points: 58 - Cost: £4.9m
Player: Murillo Santiago Costa dos Santos - Defender - Team Code: 17 - Total points: 53 - Cost: £4.6m
Player: Nikola Milenković - Defender - Team Code: 17 - Total points: 50 - Cost: £4.5m
